In [36]:
import requests
from bs4 import BeautifulSoup
import re

In [37]:
url = 'https://www.melon.com/chart/index.htm'
req_header_dict = {
    # 요청헤더 : 브라우저정보
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}
res = requests.get(url, headers=req_header_dict)
print(res.status_code)

200


In [38]:
if res.ok:
    html = res.text
    #html 전체 소스보기
    #print(html)
    soup = BeautifulSoup(html, 'html.parser')
    #html parsing
    #print(soup)  

In [39]:

#parsing한 html중 곡에대한 정보를 담음 태그만 조회!!
a_tags = soup.select("div#tb_list tr a[href*='playSong']")
print(len(soup.select("div#tb_list tr a[href*='playSong']")))  
#top100이니까 100이 나와야한다.
#print(a_tags)

100


In [40]:
# playSong이 있는 <a></a>를 enumerate하여 dict형식으로 만들어준다.
#Top 100곡의 정보를 저장할 list 선언
song_list = []
for idx, a_tag in enumerate(a_tags,1):
    #노래 1곡의 정보를 저장할 dict 선언 
    song_dict = {}
    #노래 제목 갖고오기 <a href="">노래제목</a>
    song_title = a_tag.text
    song_dict['song_title'] = song_title
    
    #print(song_dict)
    
    #a태그의 href 속성의 값을 추출하기 javascript:melon.play.playSong('1000002721',34535898);
    href_value = a_tag['href']

    #Song ID를 찾기 위한 정규표현식
    matched = re.search(r'(\d+)\);', href_value)
    #print(matched)
    if matched:
        song_id = matched.group(1) # group(0) : 34535898);  group(1) : 34535898
        song_dict['song_id'] = song_id
        #print(song_id)
        
        song_detail_url = f'https://www.melon.com/song/detail.htm?songId={song_id}' #곡정보의 url
        song_dict['song_detail_url'] = song_detail_url
        song_list.append(song_dict)

print(len(song_list))
print(song_list)

100
[{'song_title': '사랑은 늘 도망가', 'song_id': '34061322', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=34061322'}, {'song_title': '취중고백', 'song_id': '34431086', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=34431086'}, {'song_title': '호랑수월가', 'song_id': '34535898', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=34535898'}, {'song_title': '이제 나만 믿어요', 'song_id': '32508053', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=32508053'}, {'song_title': 'Step Back', 'song_id': '34494149', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=34494149'}, {'song_title': 'ELEVEN', 'song_id': '34349913', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=34349913'}, {'song_title': '그대라는 사치', 'song_id': '33652821', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=33652821'}, {'song_title': '별빛 같은 나의 사랑아', 'song_id': '33337651', 'song_detail_url': 'https://www.melon.com/song/detail.htm?s

In [41]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.melon.com/chart/index.htm'
req_header_dict = {
    # 요청헤더 : 브라우저정보
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}
res = requests.get(url, headers=req_header_dict)
print(res.status_code)


200


In [45]:
song_detail_list = []
for idx, song in enumerate(song_list,1):
    #노래 1곡의 상세정보를 저장할 dict
    song_detail_dict = {}
    song_detail_url = song['song_detail_url']

    
    res = requests.get(song_detail_url, headers=req_header_dict)
    #print(res.status_code)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        #print(idx,song['song_title'])
        song_detail_dict['곡명'] = song['song_title']
        #노래 1곡의 상세정보에 '곡명'을 찾아 주었다. 
        #다음으로 상세정보 '가수'를 찾는다
        singer = soup.select("a[href*='goArtistDetail'] span")
        #print(singer)
        if singer:
            song_detail_dict['가수'] = singer[0].text # singer리스트에서 0번째 순서에 해당하는것을 text로 뽑으면 이름이다.
        #print(singer[0].text)
        
        #다음으로 div.meta에서 dd로 끝나는 값들을 넣어준다
        song_di = soup.select("div.meta dd")
        #print(song_di)
        if song_di:
            song_detail_dict['앨범'] = song_di[0].text
            song_detail_dict['발매일'] = song_di[1].text
            song_detail_dict['장르'] = song_di[2].text
          
        
        song_id = song['song_id'] #song_id = matched.group(1) 갖고 와서 곡별 고유 id를 이용하자
        like_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_id}'
        like_res = requests.get(like_url, headers=req_header_dict)
        
        
        if like_res.ok:
           song_detail_dict['좋아요'] = like_res.json()['contsLike'][0]['SUMMCNT']
           #{'contsLike': [{'CONTSID': 34061322, 'LIKEYN': 'N', 'SUMMCNT': 124335}], 'httpDomain': 'http://www.melon.com', 'httpsDomain': 'https://www.melon.com', 'staticDomain': 'https://static.melon.co.kr'}
            #좋아요 수 추가 완료
        
        song_lyric = soup.select("div#d_video_summary")
        if song_lyric:
            song_detail_dict['가사'] = song_lyric[0].text
            pattern = re.compile(r'[\r\n\t]')
            # \r\n\t 특수문자를 ''(empty string)으로 대체(substitute)해라
            lyric = pattern.sub('', song_detail_dict['가사'].strip())
        else:
            lyric = ''
        song_detail_dict['가사'] = lyric
        
        song_detail_list.append(song_detail_dict)   
print(song_detail_list)

[]


In [43]:
import json

with open('songs.json','w',encoding='utf-8') as file:
    json.dump(song_detail_list, file)

In [44]:
## songs.json을 load 해보자 

with open('songs.json','r',encoding='utf-8') as file:
    songs_json = json.loads(file.read())
    
print(songs_json)

[]
